In [4]:
# Importing the Libraries
import pandas as pd
import numpy as np
import csv

In [5]:
PPPData = pd.read_csv('public_150k_plus_fingerprint.csv')

# Index (read) the rows in the PPPData['BorrowerNameFingerprint'] and using the size aggrigate function, count the number of entries that a unique barrower has in this column.
LoanCount = PPPData.pivot_table(index='BorrowerNameFingerprint',aggfunc='size')

# Convert this into a dataframe
LoanCountDF=LoanCount.to_frame('Loan Count')

print('Description of duplicate borrower table:')
print(LoanCountDF.describe())

Description of duplicate borrower table:
          Loan Count
count  694279.000000
mean        1.104022
std         0.306489
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        12.000000


In [6]:
# Using the processing method of the data loan, we can identify the second-round loans (PPP for first, PPS for second)

# Filter the DataFrame to include only loans where ProcessingMethod is 'PPS' (Second Round Loans) by saving only rows with said "== '__'" trait
PPSLoans = PPPData[PPPData['ProcessingMethod'] == 'PPS']

PPSLoansUnique = PPSLoans.drop_duplicates(subset='BorrowerNameFingerprint')
PPSLoansUnique
# Prints the shape of the dataframe, in the total (Row, Col) format
print('Loans labeled as second-round loans:')
print(PPSLoans.shape)

print('Loans labeled as second-round loans:')
print(PPSLoansUnique.shape)


Loans labeled as second-round loans:
(103949, 51)
Loans labeled as second-round loans:
(103947, 51)


In [7]:
''' 
We define a function that takes an input in the format (NameOfDF, ColumnInDF), and identifies 
all rows in the DF that have duplicate values in the specified column (ColumnInDF). 
The function returns a DF containing all the rows with duplicate values in that column.
'''

def find_all_rows_with_duplicates(df, column_name):
    # Count the number of occurrences of each unique value in the specified column
    value_counts = df[column_name].value_counts()
    
    # Filter the counts to get the values that appear more than once (duplicates)
    duplicate_values = value_counts[value_counts > 1].index
    
    # Step 3: Return all rows from the DataFrame where the column value is a duplicate
    return df[df[column_name].isin(duplicate_values)]  # Filtering the rows with duplicate values

find_all_rows_with_duplicates(PPSLoans, 'BorrowerNameFingerprint')


,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerNameFingerprint,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED,BusinessType,OriginatingLenderLocationID,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,Gender,Veteran,NonProfit
619062,7245898304,01/28/2021,464,PPS,ANDY REED,andy reed mount pleasant SC,624 Long Point Rd Unit A,Mount Pleasant,SC,29464-8283,...,NaN,NaN,Limited Liability Company(LLC),203685,Pinnacle Bank,NASHVILLE,TN,Unanswered,Unanswered,NaN
619972,6117878310,01/26/2021,464,PPS,ANDY REED,andy reed mount pleasant SC,1117 Park West Blvd Ste E,Mount Pleasant,SC,29466-7120,...,NaN,NaN,Limited Liability Company(LLC),203685,Pinnacle Bank,NASHVILLE,TN,Unanswered,Unanswered,NaN
682349,2053668308,01/20/2021,610,PPS,LNF (SOL) OPERATING CORP,corp lnf operating dallas TX,6030 Luther Ln Ste 180,Dallas,TX,75225-6151,...,NaN,NaN,Corporation,69884,"Dallas Capital Bank, National Association",DALLAS,TX,Unanswered,Unanswered,NaN
699232,2041138305,01/20/2021,610,PPS,LNF (PR) OPERATING CORP,corp lnf operating dallas TX,6030 Luther Ln Ste 180,Dallas,TX,75225-6151,...,NaN,NaN,Corporation,69884,"Dallas Capital Bank, National Association",DALLAS,TX,Unanswered,Unanswered,NaN


In [8]:
# Write the columns from PPSLoans into the PPSLoansSubset DF
PPSLoansSubset = PPSLoansUnique[['BorrowerName', 'BorrowerState', 'CurrentApprovalAmount', 'JobsReported']]

# Rename the columns in PPSLoansSubset (in the order which they come,) as the designated name.
PPSLoansSubset.columns = ['Company Name', 'Company State', '2nd Loan Amount', 'Jobs Reported']

# Calculate the dollars per job
DollarsPerJob = PPSLoansSubset['2nd Loan Amount']/PPSLoansSubset['Jobs Reported']

# Insert the newly calculated Dollars Per Job value for each business as a new column in the format of (ColumNumber, 'ColumnTitle', DFImported)
PPSLoansSubset.insert(4, 'Dollars per Job', DollarsPerJob)
SortedPPSLoansSubset = PPSLoansSubset.sort_values(by='Company Name')

# Making the csv
SortedPPSLoansSubset.to_csv('second_loans.csv', index=False)
SortedPPSLoansSubset

,Company Name,Company State,2nd Loan Amount,Jobs Reported,Dollars per Job
679924,#1 QUALITY ELECTRIC INC.,TX,225000.00,18.0,12500.000000
253653,#3 ZIMMIES INC.,IL,405667.00,17.0,23862.764706
256305,#7 ZIMMIES INC.,IL,337456.00,13.0,25958.153846
401232,#MYBLUESKY LLC,MO,305171.77,31.0,9844.250645
739171,&YET LLC,WA,262248.00,13.0,20172.923077
...,...,...,...,...,...
423032,ZZ HOME CARE LLC,NC,321217.50,59.0,5444.364407
599178,ZZH INC.,PA,249003.93,73.0,3411.012740
755743,ZZIP STOP INC.,WI,264778.50,99.0,2674.530303
103020,ZZM DENTAL MANAGEMENT INC,CA,203105.00,25.0,8124.200000
